<a href="https://colab.research.google.com/github/mraniketr/DeepLearningProjects/blob/master/Sentiment_emoji_prediction2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
!pip install emoji
import emoji
import matplotlib.pyplot as plt

import csv
import pandas as pd
from sklearn.metrics import confusion_matrix

from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform

%matplotlib inline

In [0]:

def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()


def read_csv(filename = '/content/drive/My Drive/COLAB DATA/Emojify/train_emoji.csv'):
    phrase = []
    emoji = []

    with open (filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile)

        for row in csvReader:
            phrase.append(row[0])
            emoji.append(row[1])

    X = np.asarray(phrase)
    Y = np.asarray(emoji, dtype=int)

    return X, Y

def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y


emoji_dictionary = {"0": "\u2764\uFE0F",    #love
                    "1": "",          #neutral
                    "2": ":smile:",     #happy
                    "3": ":disappointed:", #sad
                    "4": ":pouting_face:"}   #angry

def label_to_emoji(label):
    return emoji.emojize(emoji_dictionary[str(label)], use_aliases=True)
              
    
def print_predictions(X, pred):
    print()
    for i in range(X.shape[0]):
        print(X[i], label_to_emoji(int(pred[i])))
        
        
def plot_confusion_matrix(y_actu, y_pred, title='Confusion matrix', cmap=plt.cm.gray_r):
    
    df_confusion = pd.crosstab(y_actu, y_pred.reshape(y_pred.shape[0],), rownames=['Actual'], colnames=['Predicted'], margins=True)
    
    df_conf_norm = df_confusion / df_confusion.sum(axis=1)
    
    plt.matshow(df_confusion, cmap=cmap) # imshow
    #plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(df_confusion.columns))
    plt.xticks(tick_marks, df_confusion.columns, rotation=45)
    plt.yticks(tick_marks, df_confusion.index)
    #plt.tight_layout()
    plt.ylabel(df_confusion.index.name)
    plt.xlabel(df_confusion.columns.name)
    
    
def predict(X, Y, W, b, word_to_vec_map):
   
    m = X.shape[0]
    pred = np.zeros((m, 1))
    
    for j in range(m):                       # Loop over training examples
        
        words = X[j].lower().split()
        
        avg = np.zeros((50,))
        for w in words:
            avg += word_to_vec_map[w]
        avg = avg/len(words)

        Z = np.dot(W, avg) + b
        A = softmax(Z)
        pred[j] = np.argmax(A)
        
    print("Accuracy: "  + str(np.mean((pred[:] == Y.reshape(Y.shape[0],1)[:]))))
    
    return pred

In [0]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('/content/drive/My Drive/COLAB DATA/Emojify/glove.6B.50d.txt')


In [0]:
X_train, Y_train = read_csv('/content/drive/My Drive/COLAB DATA/Emojify/neds.csv')
X_test, Y_test = read_csv('/content/drive/My Drive/COLAB DATA/Emojify/tesss.csv')


In [0]:
maxLen = len(max(X_train, key=len).split())

In [0]:


def sentences_to_indices(X, word_to_index, max_len):
    m = X.shape[0]                         
    X_indices = np.zeros((m,max_len))
    for i in range(m):                               
        sentence_words = [i.lower() for i in X[i].split()]
        j = 0
        for w in sentence_words:
            X_indices[i, j] = word_to_index[w]
            j = j+1
      
    
    return X_indices

In [0]:
Y_oh_train = convert_to_one_hot(Y_train, C = 5)
Y_oh_test = convert_to_one_hot(Y_test, C = 5)

In [0]:
idx = 50

In [0]:

def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    vocab_len = len(word_to_index) + 1                  
    emb_dim = word_to_vec_map["cucumber"].shape[0]      
    emb_matrix = np.zeros((vocab_len, emb_dim))
    for word, idx in word_to_index.items():
        emb_matrix[idx, :] = word_to_vec_map[word]
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    embedding_layer.build((None,)) 
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [0]:

def Emojify_V2(input_shape, word_to_vec_map, word_to_index):
    sentence_indices = Input(input_shape, dtype='int32')
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    embeddings = embedding_layer(sentence_indices)  
    X = LSTM(128, return_sequences = True)(embeddings)
    X = Dropout(0.5)(X)
    X = LSTM(128, return_sequences=False)(X)
    X = Dropout(0.5)(X)
    X = Dense(5)(X)
    X = Activation('softmax')(X)
    model = Model(inputs=sentence_indices, outputs=X)
    
    return model

In [15]:
model = Emojify_V2((maxLen,), word_to_vec_map, word_to_index)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 11)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 11, 50)            20000050  
_________________________________________________________________
lstm_1 (LSTM)                (None, 11, 128)           91648     
_________________________________________________________________
dropout_1 (Dropout)          (None, 11, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 645 

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 5)

In [22]:
model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32, shuffle=True)

Epoch 1/50
132/132 [==============================] - 1s 7ms/step - loss: 1.6048 - accuracy: 0.2197
Epoch 2/50
132/132 [==============================] - 0s 1ms/step - loss: 1.5475 - accuracy: 0.3258
Epoch 3/50
132/132 [==============================] - 0s 1ms/step - loss: 1.5202 - accuracy: 0.3485
Epoch 4/50
132/132 [==============================] - 0s 1ms/step - loss: 1.4735 - accuracy: 0.3788
Epoch 5/50
132/132 [==============================] - 0s 1ms/step - loss: 1.4471 - accuracy: 0.3939
Epoch 6/50
132/132 [==============================] - 0s 1ms/step - loss: 1.3868 - accuracy: 0.4773
Epoch 7/50
132/132 [==============================] - 0s 1ms/step - loss: 1.3596 - accuracy: 0.3939
Epoch 8/50
132/132 [==============================] - 0s 1ms/step - loss: 1.2100 - accuracy: 0.5076
Epoch 9/50
132/132 [==============================] - 0s 1ms/step - loss: 1.1469 - accuracy: 0.5076
Epoch 10/50
132/132 [==============================] - 0s 1ms/step - loss: 1.0828 - accuracy: 0.5833

In [31]:

x_test = np.array(['how are you','funny lol','i hate you','not feeling happy','i love python'])
for i in range(0,len(x_test)):
  X_test_indices = sentences_to_indices(np.array([x_test[i]]), word_to_index, maxLen)
  print(x_test[i] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices))))  

#No Emoji - Neutral
#Happy,Angry,Sad,Love


how are you 
funny lol 😄
i hate you 😡
not feeling happy 😞
i love python ❤️
